In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

### 문제 1
- 주소 (https://moon9303.pythonanywhere.com/open_api)
- servicekey는 asdf1234
- 해당 주소에서 받은 데이터를 데이터프레임으로 생성하시오

In [2]:
url = 'https://moon9303.pythonanywhere.com/open_api'

# 호출
res = requests.get(url,{"servicekey": "asdf1234"})

res_data = res.json()
# print(res_data)
# type(res_data)

df = pd.DataFrame(res_data)
df

,기말,리포트,중간,퀴즈,학번
0,95,20,90,20,S01
1,83,18,82,18,S02
2,78,18,80,18,S03
3,75,10,78,10,S04
4,91,12,93,12,S05
5,75,16,71,16,S06
6,80,18,60,18,S07
7,65,14,72,14,S08
8,65,14,65,14,S09
9,78,10,85,10,S10


### 문제2
- 주소 (https://moon9303.pythonanywhere.com/)
- 해당 주소에서 table 태그를 크롤링하여 데이터프레임으로 생성하시오

In [3]:
url2= 'https://moon9303.pythonanywhere.com/'
res2 = requests.get(url2)
data = res2.text
soup = bs(data,'html.parser')
table_data = soup.find_all('table')[0]

# columns 추출하기
thead_data = table_data.find('thead')
th_list = thead_data.find_all('th')

cols = list(
    map(
        lambda x:x.get_text(),
        th_list
    )
)

# rows 추출하기
tbody_data = table_data.find('tbody')
tr_list = tbody_data.find_all('tr')
tr_data = tr_list[0]
val_list = tr_data.find_all(['th','td'])

values = []
for tr_data in tr_list:
    val_list = tr_data.find_all(['th','td'])
    value = []
    for val in val_list:
        value.append(val.get_text().strip())
    values.append(value)
    
# cols와 values를 가지고 DataFrame 생성
df = pd.DataFrame(values,columns = cols)
df

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,virginica
146,6.3,2.5,5.0,1.9,virginica
147,6.5,3.0,5.2,2.0,virginica
148,6.2,3.4,5.4,2.3,virginica


### 문제 3
- csv 폴더 안에 있는  titanic 파일을 로드
- 해당하는 데이터에 구조를 파악
- 컬럼 별 결측치의 개수를 파악
### 문제 4
- Survived, Pclass 컬럼의 데이터들을 문자형 데이터로 변경하시오
### 문제 5
- Pclass별로 Survived 값(0, 1) 개수를 출력하시오

In [4]:
# 문제 3
df = pd.read_csv('./csv/titanic.csv')
df.info() # 데이터의 타입 확인
df.describe() # 기본 통계 확인
df.isna() # 결측치 확인 -> Age 177개, Cabin687개, Embarked 2개 결측치 존재
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
# 문제 4

# 변환 전 데이터 타입 확인
print('변환 전 데이터 타입 확인 : ')
print(type(df['Survived'][0]))
print(type(df['Pclass'][0]))
#<class 'numpy.int64'> 출력 
#<class 'numpy.int64'> 출력

# 문자열 변환
df[['Survived', 'Pclass']] = df[['Survived', 'Pclass']].applymap(str)

# 문자열 변변환 후 데이터 타입 확인
print('문자열 변환 후 데이터 타입 확인 : ')
print(type(df['Survived'][0]))
print(type(df['Pclass'][0]))
# <class 'str'> 출력 
# <class 'str'>출력 

변환 전 데이터 타입 확인 : 
<class 'numpy.int64'>
<class 'numpy.int64'>
문자열 변환 후 데이터 타입 확인 : 
<class 'str'>
<class 'str'>


In [6]:
# 5번
df.groupby('Pclass')[['Survived']].value_counts().unstack()

Survived,0,1
Pclass,,
1,80,136
2,97,87
3,372,119


### 문제 6
- csv 폴더에서 airquality 파일을 로드 
- 데이터에서 결측치의 개수를 확인하고 가장 많은 결측치를 가진 컬럼의 결측치를 0으로 대체하시오
- 결측치를 제외한 평균과 결측치를 0으로 대체한 뒤 평균 값을 구하여 차이를 출력하시오
### 문제 7
- Wind 컬럼의 데이터를 min-max 스케일링을 이용하여 스케일링 작업을 하시오
- 스케일링 후 평균 값을 출력하시오
### 문제 8
- 월별 평균 기온을 구하시오

In [7]:
# 6번
df = pd.read_csv('./csv/airquality.csv')
df.isna().sum() # 결측치는 Ozone 37개, Solar.R 7개 존재, 가장 많은 결측치를 가진 컬럼은 Ozone
df


# 결측치를 제외한 데이터프레임을 위한 조건
flag = df.isna().any(axis='columns')
# 결측치를 제외한 평균값
print(f'결측치를 제외한 평균값 : {df.loc[~flag]['Ozone'].mean()}')  # 42.0990990990991
non_null = df.loc[~flag]['Ozone'].mean()


# 가장 많은 결측치를 가진 Ozone 컬럼의 결측치를 0으로 대체한 뒤 평균값
print(f'결측치를 0으로 대체한 뒤 평균값 :{df['Ozone'].fillna(0).mean()}') # 31.941176470588236
null_0 = df['Ozone'].fillna(0).mean()



print(f'결측치 제외한 평균과 0으로 대체한 평균의 차이 :  {non_null - null_0}')

결측치를 제외한 평균값 : 42.0990990990991
결측치를 0으로 대체한 뒤 평균값 :31.941176470588236
결측치 제외한 평균과 0으로 대체한 평균의 차이 :  10.157922628510864


In [8]:
# 7번
from sklearn.preprocessing import MinMaxScaler

# 스케일링 전 wind 평균
print(f'스케일링 전 wind 평균 : {df['Wind'].mean()}')

# 객체 생성성
MmScaler = MinMaxScaler()

# fit과 동시에 스케일링
scaled_wind = MmScaler.fit_transform(df[['Wind']])

# 스케일링된 wind 평균균
print(f'스케일링 된 wind 평균 : {scaled_wind.mean()}')

스케일링 전 wind 평균 : 9.957516339869281
스케일링 된 wind 평균 : 0.4346061231510147


In [9]:
# 8번
# 월별 평균 기온 
df.groupby('Month')[['Temp']].mean()

,Temp
Month,
5,65.548387
6,79.100000
7,83.903226
8,83.967742
9,76.900000


### 문제 9
- csv 폴더에서 HR-Employee-Attrition 파일을 로드 
- Attrition 컬럼의 데이터에서 Yes는 1로 No는 0으로 데이터를 변경
- 해당 데이터의 빈도수를 출력하시오
### 문제 10
- 데이터 중 int64로 이루어진 컬럼들만 따로 출력하여 저장하시오


In [10]:
# 9번

# 데이터 로드
df = pd.read_csv('./csv/HR-Employee-Attrition.csv')

# 혹시나 Yes,No가 아닌 다른 값이 있나 확인
df['Attrition'].value_counts()

# Yes는 1로, no는 0으로 변환
for i in range(len(df)):
    if df['Attrition'][i] == 'Yes':
        df['Attrition'][i] = 1
    else:
        df['Attrition'][i] = 0
# df['Attrition'] = df['Attrition'].map({'Yes': 1, 'No': 0})

# 해당 데이터 빈도수 출력
df['Attrition'].value_counts()

Attrition
0    1233
1     237
Name: count, dtype: int64

In [11]:
# 10 번

# 컬럼의 데이터 타입 확인
# df.info()

# ['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']
int_64_cols = []
for col in df.columns:
    if df[col].dtype == 'int64':
        int_64_cols.append(col)
print('int_64타입의 컬럼명들 : ')
print(int_64_cols)

# int_64 컬럼들로 이루어진 데이터프레임
print('int_64 컬럼들로 이루어진 데이터프레임 :')
df[int_64_cols]

int_64타입의 컬럼명들 : 
['Age', 'DailyRate', 'DistanceFromHome', 'Education', 'EmployeeCount', 'EmployeeNumber', 'EnvironmentSatisfaction', 'HourlyRate', 'JobInvolvement', 'JobLevel', 'JobSatisfaction', 'MonthlyIncome', 'MonthlyRate', 'NumCompaniesWorked', 'PercentSalaryHike', 'PerformanceRating', 'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel', 'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance', 'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion', 'YearsWithCurrManager']
int_64 컬럼들로 이루어진 데이터프레임 :


,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,1102,1,2,1,1,2,94,3,2,...,1,80,0,8,0,1,6,4,0,5
1,49,279,8,1,1,2,3,61,2,2,...,4,80,1,10,3,3,10,7,1,7
2,37,1373,2,2,1,4,4,92,2,1,...,2,80,0,7,3,3,0,0,0,0
3,33,1392,3,4,1,5,4,56,3,1,...,3,80,0,8,3,3,8,7,3,0
4,27,591,2,1,1,7,1,40,3,1,...,4,80,1,6,3,3,2,2,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1465,36,884,23,2,1,2061,3,41,4,2,...,3,80,1,17,3,3,5,2,0,3
1466,39,613,6,1,1,2062,4,42,2,3,...,1,80,1,9,5,3,7,7,1,7
1467,27,155,4,3,1,2064,2,87,4,2,...,2,80,1,6,0,3,6,2,0,3
1468,49,1023,2,3,1,2065,4,63,2,2,...,4,80,0,17,3,2,9,6,0,8
